# Тема 8: Подготовка признаков временных рядов для ML
## Теория: почему подготовка признаков критически важна
Традиционные ML‑модели (Linear Regression, RandomForest, XGBoost и др.) **не умеют работать с временной зависимостью напрямую**.
Им нужно вручную подготовить входные признаки, которые будут отражать динамику ряда.

**Основные типы признаков временных рядов:**
- **Лаги (lags)** — значения ряда в предыдущие моменты времени. Это ключевой способ дать модели память.
- **Скользящие окна (rolling mean/std/min/max)** — агрегаты локального поведения.
- **Экспоненциальное сглаживание (EWM)** — важно для учёта последних значений.
- **Календарные признаки** — hour, dayofweek, праздник/рабочий день, квартал, месяц.

ML‑модели обучаются по признакам, поэтому качество прогнозов напрямую зависит от качества фичей.

## 1. Генерация синтетического телеком‑ряда

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)
n = 200
t = np.arange(n)

# components
trend = 0.1 * t
daily = 5 * np.sin(2*np.pi*t/24)
weekly = 3 * np.sin(2*np.pi*t/168)
noise = np.random.normal(0, 2, n)

traffic = 100 + trend + daily + weekly + noise

# dataframe
df = pd.DataFrame({
    'time': pd.date_range('2023-01-01', periods=n, freq='H'),
    'traffic': traffic
}).set_index('time')
df.head()

## 2. Визуализация ряда

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(df['traffic'])
plt.title('Синтетический телеком-трафик')
plt.show()

## 3. Создание признаков

In [ ]:
def add_lags(data, col, lags):
    for l in lags:
        data[f'{col}_lag{l}'] = data[col].shift(l)
    return data

df = add_lags(df.copy(), 'traffic', list(range(1,25)))
df['rolling_mean_6'] = df['traffic'].rolling(6).mean()
df['rolling_std_6'] = df['traffic'].rolling(6).std()
df['ewm_12'] = df['traffic'].ewm(span=12).mean()

# date features
df['hour'] = df.index.hour
df['dayofweek'] = df.index.dayofweek
df['is_weekend'] = (df.index.dayofweek >= 5).astype(int)

# drop NaN
df = df.dropna()
df.head()

## 4. Корреляционный анализ

In [ ]:
plt.figure(figsize=(12,10))
plt.imshow(df.corr(), cmap='coolwarm')
plt.colorbar()
plt.title('Корреляции признаков')

plt.show()

## 5. Задания для студентов
**Задание 1.** Добавить признаки:
- rolling_min_12
- rolling_max_12
- ewm_3, ewm_24

**Задание 2.** Выполнить STL‑разложение ряда и добавить сезонные компоненты как фичи.

**Задание 3.** Построить heatmap зависимостей *только* по лаговым признакам.

**Задание 4.** Создать функцию `generate_features(df, config)` которая автоматизирует построение фичей.

**Задание 5. Мини‑проект.**
Создать *3 набора признаков*:
1. только лаги,
2. лаги + окна,
3. все признаки.

Сравнить их по MAE с моделью LinearRegression или RandomForest.
